In [2]:
#import dependencies 
import pandas as pd 
import requests 
from sqlalchemy import create_engine
import psycopg2
import config 

# Data 

In [20]:
#get sample of data -1000 rows
endpoint = "https://data.sfgov.org/resource/imvp-dq3v.json?$limit=1000"
r = requests.get(url=endpoint)

In [28]:
#convert to datframe 
data_df = pd.DataFrame(r.json())

In [29]:
data_df.shape

(1000, 8)

In [30]:
#review dataframe 
data_df.head()

,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt
0,135482724_4_09302017104221,352-05200,CASTRO ST 500,CREDIT CARD,2017-09-30T10:42:21.000,2017-09-30T11:02:21.000,NS,0.75
1,135482725_4_09302017104224,418-08670,FOLSOM ST 800,CREDIT CARD,2017-09-30T10:42:24.000,2017-09-30T12:42:17.000,NS,5.54
2,135482726_4_09302017104223,568-45310,MISSION ST 4500,CASH,2017-09-30T10:42:23.000,2017-09-30T11:26:07.000,NS,0.75
3,135482727_4_09302017104226,350-24090,CALIFORNIA ST 2400,CASH,2017-09-30T10:42:26.000,2017-09-30T11:27:56.000,AT,0.15
4,135482728_4_09302017104226,226-34480,26TH ST 3400,CASH,2017-09-30T10:42:26.000,2017-09-30T11:42:26.000,NS,2.25


In [37]:
[i for i in data_df.columns]

['transmission_datetime',
 'post_id',
 'street_block',
 'payment_type',
 'session_start_dt',
 'session_end_dt',
 'meter_event_type',
 'gross_paid_amt']

In [39]:
data_df.meter_event_type.unique()

array(['NS', 'AT'], dtype=object)

In [67]:
len(data_df.street_block.unique())

534

In [64]:
pd.DataFrame(data_df.street_block.unique())

,0
0,CASTRO ST 500
1,FOLSOM ST 800
2,MISSION ST 4500
3,CALIFORNIA ST 2400
4,26TH ST 3400
...,...
529,TREAT AVE 1100
530,03RD ST 700
531,BAY ST 400
532,POLK ST 800


# Database 

In [71]:
#connect to postgres database 
conn = psycopg2.connect(database="donniedata", user=config.db_user, password=config.db_password, sslmode="disable")
cur = conn.cursor();
conn.autocommit = True

In [ ]:
sf_ticket_trans

In [73]:
[i for i in data_df.columns]

['transmission_datetime',
 'post_id',
 'street_block',
 'payment_type',
 'session_start_dt',
 'session_end_dt',
 'meter_event_type',
 'gross_paid_amt']

In [78]:
sql = '''
CREATE TABLE sf_ticket_trans.fact_transactions ( 
transmission_datetime TEXT,
payment_type_id INTEGER,
street_block_id INTEGER,
post_id TEXT,
meter_event_type TEXT,
gross_paid_amt DECIMAL,
session_start_dt TIMESTAMP,
session_end_date TIMESTAMP
)
'''

cur.execute(sql)